In [ ]:
'''
This code is not ready! I still have to add comments and some variable names *need* changing!
'''

import networkx as nx
import random
import copy
import time
import matplotlib.pyplot as plt
import sys
%matplotlib inline

def progressBar(value, endvalue, bar_length=20):

        percent = float(value) / endvalue
        arrow = '-' * int(round(percent * bar_length)-1) + '>'
        spaces = ' ' * (bar_length - len(arrow))

        sys.stdout.write("\rCalculating: [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))
        sys.stdout.flush()

        
def kawai_draw(G, infected=[]) :
    colors = []
    for node in G :
        if node in infected :
            colors.append('red')
        else :
            colors.append('green')
    nx.draw(G, layout=nx.kamada_kawai_layout(G), node_color=colors, node_size = 30, alpha = 0.7)
    plt.show()
    
#G = nx.scale_free_graph(1000)

G = nx.scale_free_graph(1000)
G = G.to_undirected()

In [ ]:
kawai_draw(G)

In [ ]:
def generate_egos(G, node, max_radius):
    resulting_G = []
    
    G_ego = nx.ego_graph(G, node, radius=1)
    if len(G_ego) > 1 :
        resulting_G.append(G_ego)
        G_ego_previous = G_ego

        for i in range(2, max_radius+1) :
            G_ego = nx.ego_graph(G, node, radius=i)
            G_ego_toremove = copy.deepcopy(G_ego)
            G_ego_toremove.remove_nodes_from(G_ego_previous)
            G_ego_previous = G_ego

            if len(G_ego_toremove) > 0 :
                resulting_G.append(G_ego_toremove)
            else :
                break

        resulting_G[0].remove_node(node)
    return resulting_G

def generate_egos_allnodes(G, max_radius) :
    result = []
    
    nodecount = 1
    for node in G :
        progressBar(nodecount, len(G), 50)
        result.append(generate_egos(G, node, max_radius))
        nodecount +=1
        
    return result

radius_split = generate_egos_allnodes(G, 10)

In [ ]:
def infected_percentage(G, infected) :
    
    count = 0
    
    for node in G :
        if node in infected :
            count += 1
        
    return count / len(G)

def infected_average_by_radius(G, Gs_list, radius, infected) :
    node_count = []
    percentage_sum = []
    for _ in range(0, radius) :
        node_count.append(0)
        percentage_sum.append(0)
            
    current_node = 0
    for node in Gs_list :
        if current_node in infected :
            current_radius = 0
            for item in node :
                percentage_sum[current_radius] += infected_percentage(item, infected)
                node_count[current_radius] += 1
                current_radius += 1
        current_node += 1
        
    average_by_radius = []
    for i in range(0, radius) :
        if node_count[i] > 0 :
            average_by_radius.append(percentage_sum[i] / node_count[i])
        
    return average_by_radius


In [ ]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics.SIModel as si

# Model selection
model = si.SIModel(G)

# Model Configuration
cfg = mc.Configuration()
cfg.add_model_parameter('beta', 0.01) # prob of infecteded
cfg.add_model_parameter("percentage_infected", 0.5) # initially infected  %
model.set_initial_status(cfg)

# Simulation execution
iterations = model.iteration_bunch(40)

In [ ]:
infected = []
for it in iterations :
    for node, i in it['status'].items() :
        if i == 1:
            infected.append(node)
            
    print('Infected count', len(infected))
    #kawai_draw(G)    
    print(infected_average_by_radius(G, radius_split, 10, infected))

In [ ]:
'''
node = 188
for i in range(0, len(radius_split[node])) :
    
    print("Iteração", i+1)
    temp = nx.ego_graph(G, node, radius=i+1)
    temp.remove_nodes_from([node])
    print("Percentage", infected_percentage(temp, infected))
    kawai_draw(temp, infected)
    for x in temp :
        if x in infected :
            print(x)
'''